The purpose of this notebook is to assess the performance of `AMBER` netcdf read/write operations in the `MDAnalysis` package as it pertains to the decision to replace `netCDF4` with `scipy.io.netcdf`